In [1]:
import os

# Change working dir so that we can import local module
working_dir = os.path.join(os.getcwd(), os.pardir)
os.chdir(working_dir)
from reshpc import nersc_interface as rn
print('Working directory {}'.format(working_dir))

Working directory /home/john/projects/resonant-hpc/git/resonantrpc/dev/jupyter/notebooks/..


In [3]:
nersc = rn.NerscInterface()
nersc.login()

sending command...


'OK'

In [4]:
# Generate new folder name yymmdd_hhmm
import datetime
now = datetime.datetime.now()
datetime_code = now.strftime('%y%m%d-%H%M')
datetime_code

'201021-114130'

In [5]:
# Generate path on Cori $SCRATCH folder
scratch_folder = nersc.get_scratch_folder()
nersc_folder = '{}/reshpc/demo01/{}'.format(scratch_folder, datetime_code)
nersc_folder

sending command...


'/global/cscratch1/sd/johnt/reshpc/201021-114130'

In [6]:
nersc.make_folder(nersc_folder)

sending command...


'OK'

In [7]:
# Upload demo 01 baseline file
xml_file = 'demo_01.xml'
local_file = os.path.join(working_dir, 'data', xml_file)
assert os.path.exists(local_file)

nersc.upload_file(local_file, nersc_folder)

sending command...


'OK'

In [8]:
# Specify SLURM script
ats = '/project/projectdirs/m2398/ideas/ats/install/cori/ats-0.88-basic/RelWithDebInfo/PrgEnv-gnu-6.0.5/bin/ats'
timeout_min = 5
nodes = 1
cores_per_node = 1

slurm_commands = [
    '#!/bin/bash',
    '#SBATCH --account=m2398',
    '#SBATCH --chdir={}'.format(nersc_folder),
    '#SBATCH --partition=debug',
    '#SBATCH --time=0:{}:0'.format(timeout_min),
    '#SBATCH --nodes={}'.format(nodes),
    '#SBATCH --tasks-per-node={}'.format(cores_per_node),
    '#SBATCH --constraint=haswell',
    'ulimit -s unlimited',
    'srun {} --xml_file={}'.format(ats, xml_file),
    ''
]
slurm_string = '\n'.join(slurm_commands)

In [9]:
# Submit job
result = nersc.submit_job(slurm_string, nersc_folder)
print(result)

uploading slurm script...
submitting job...
{'error': '', 'status': 'OK', 'jobid': '35369711'}


In [10]:
job_id = result.get('jobid')
if job_id:
    info = nersc.get_job_info(job_id)
    print(info)
    status = info.get('status')
    print('Job Status: {}'.format(status))


{'user': 'johnt', 'nodes': '1', 'status': 'R', 'procs': '64', 'submittime': '2020-10-21T08:41:59', 'qos': 'debug_hsw', 'hostname': 'cori', 'timereq': '00:05:00', 'source': 'sacct', 'repo': 'm2398', 'queue': 'debug_hsw', 'memory': '', 'name': 'job.sbatch', 'timeuse': '00:04:16', 'jobid': '35369711'}
Job Status: R


In [12]:
import time
# Loop while status is pending, running, or completing
# https://slurm.schedmd.com/squeue.html#lbAF is a list of status codes
while status in ['PD', 'R', 'CG']:
    time.sleep(5.0)
    info = nersc.get_job_info(job_id)
    status = info.get('status')
    print(status)

CD
